In [1]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp, angle, abs, unwrap, log10
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [8.00, 4.5]
plt.rcParams["figure.autolayout"] = True
plt.rcParams["axes.grid"] = True
plt.rcParams["axes.xmargin"] = 0.0

%matplotlib inline

<img src="Figurer/NTNU_Logo.png" align="left" width="30%">
<br clear="all" />
<br></br>

# Diskretisering time 3 -  *diskrete LTI-system i tilstandsrommet*

* **Emne: AIS2102 - Dynamiske system**
* ***dato***: 3. februar 2025
* **Foreleser: Kai Erik Hoff**

# Differensialligning tidskontinuerlige system

# Tilstandsrom for tidskontinuerlige system


* Ligninger:

$$\begin{align}
\boldsymbol{\dot{x}}(t) &= \boldsymbol{A}\cdot \boldsymbol{x}(t) + \boldsymbol{B}\cdot \boldsymbol{u}(t)\\
\boldsymbol{y}(t) &= \boldsymbol{C}\cdot \boldsymbol{x}(t) + \boldsymbol{D}\cdot \boldsymbol{u}(t)
\end{align}$$

# Differanseligning for tidsdiskrete system

# Tilstandsrom for tidsdiskrete system

* Ligninger:
$$\begin{align}
\boldsymbol{x}[k+1] &= \boldsymbol{A}\cdot \boldsymbol{x}[k] + \boldsymbol{B}\cdot \boldsymbol{u}[k] \\
\boldsymbol{y}[k] &= \boldsymbol{C}\cdot \boldsymbol{x}[k] + \boldsymbol{D}\cdot \boldsymbol{u}[k]
\end{align}$$

[FIGUR]